# 量子态跨平台保真度估计

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

直接保真度估计（Direct Fidelity Estimation，DFE）能够直接估计理想量子态和实际量子态的保真度。但是，它也存在着两个问题，一方面是，当理想量子态未知时则不适用，另一方面，它不适用于混态的保真度估计。跨平台估计能够很好得解决这两个问题。

本教程将介绍跨平台估计的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论

下面，我们将对如何利用跨平台估计来估计纯态的保真度展开介绍，混态的估计可以参考论文 [1] 。我们定义两个量子平台， $\mathcal{S}_1$ 和 $\mathcal{S}_2$ ，以及量子电路 $\mathcal{U}_1$ 和 $\mathcal{U}_2$ ，这两个电路用于制备相同的 $n$ 比特量子态 $\rho_1$ 和 $\rho_2$ 。我们可以通过一下流程来估计保真度 ${\rm Tr}[\rho_1 \rho_2]$ 。

1. 随机选择酉矩阵 $U = \otimes_{i=1}^n U_i$ ，其中 $U_i$ 是从 unitary 2-design 单比特门集合中采样得到的。
2. 在 $\mathcal{S}_1$ 和 $\mathcal{S}_2$ 上分别执行计算基投影测量。
3. 利用测量结果，估计如下变量$${\rm Pr}(x, y) = {\rm Tr}\left[\left(U\otimes U \rho_1\otimes\rho_2 U^\dagger\otimes U^\dagger\right) |x\rangle\langle x|\otimes|y\rangle\langle y|\right],$$ 其中 $x,y$ 分别为量子平台 $\mathcal{U}_1, \mathcal{U}_2$ 的测量结果。
4. 重复步骤 1 至步骤 3 $K$ 次，将第 $k$ 次第概率分布记录为 ${\rm Pr}^{(k)}(x, y)$ ，并且计算平均概率分布 $$\overline{{\rm Pr}(x, y)} = \frac{1}{K}\sum_k {\rm Pr}^{(k)}(x, y). $$
5. 利用如下式子估计保真度  $${\rm Tr}(\rho_1\rho_2)=2^n\sum_{x,y}(-2)^{-\mathcal{D}[x,y]}\overline{{\rm Pr}(x, y)}, $$ 其中 $\mathcal{D}[x,y]$ 表示为字符串 $x, y$ 的汉明距离（Hamming distance）。

对应的流程图如下所示。

![CPE](./figures/cpe-chart.png "Figure 1: We complete CPE according to this flow chart.")


## 实践

下面，我们将演示如何应用跨平台估计。

首先，我们导入一些必要的包。

In [2]:
import QCompute
from Extensions.QuantumErrorProcessing.qcompute_qep.estimation.cpe_state import QuantumSnapshot, CPEState

然后，我们指定我们感兴趣的量子平台。这里，我们选择理想模拟器的第一、第二个比特作为两个不同的平台。除了下面这种构造 `QuantumSnapshot` 的方式，我们还支持利用 `read_quantum_snapshot()` 方法，从文件中读取已保存的量子平台的信息。


In [3]:
ideal_baidu1 = QuantumSnapshot(qc_name='Baidu ideal1',
                               qc=QCompute.BackendName.LocalBaiduSim2,
                               qubits=[0])
ideal_baidu2 = QuantumSnapshot(qc_name='Baidu ideal2',
                               qc=QCompute.BackendName.LocalBaiduSim2,
                               qubits=[1])

dev_list = [ideal_baidu1, ideal_baidu2]

接着，我们将设置量子程序，用于制备感兴趣的量子态。剩下的步骤就很简单了，只需要调用我们的 `CPEState` 模块则可完成直接保真度估计。这里，我们设置参数 $K=100$ ，并对结果进行可视化。

In [4]:
# Construct the circuit used to prepare target state
qp = QCompute.QEnv()
qp.Q.createList(1)

est = CPEState()
result = est.estimate(dev_list, qp, samples=100, shots=50, show=True)


最后，我们可以选择将量子平台的信息以文件的形式保存。

In [ ]:
ideal_baidu1.save_data()
ideal_baidu2.save_data()

## 参考资料

[1] Elben, Andreas, et al. "Cross-platform verification of intermediate scale quantum devices." [Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.124.010504) 124.1 (2020): 010504.